In [1]:
import h5py

import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os, pdb

sys.path.append('..')
from gtv import GraphTotalVariance

In [2]:
# Generate data to test the GTV. Use a block correlation structure and a small number of features
from utils import gen_covariance, gen_beta, gen_data
from sklearn.linear_model import Lasso

In [3]:
cov = gen_covariance('block', n_features = 30, block_size = 5, correlation=0.5)
beta = gen_beta(n_features = 30, block_size = 5, sparsity = 0.5)
X, X_test, y, y_test = gen_data(n_samples = 100, n_features = 30, kappa = 0.3, covariance = cov, beta = beta)

In [4]:
from sklearn.metrics import r2_score

In [ ]:
# Test on a 3D grid of tuning parameters
lambda_S = np.linspace(0, 1, num = 10)
lambda_TV = np.linspace(0, 1, num = 10)
lambda_1 = np.linspace(0, 1, num = 10)

r2_scores = np.zeros((10, 10, 10))
sa = np.zeros((10, 10, 10))

# Compare runtimes between quadprog and lbfgs minimizer
for i1, l1 in enumerate(lambda_S):
    for i2, l2 in enumerate(lambda_TV):
        for i3, l3 in enumerate(lambda_1):
            gtv1 = GraphTotalVariance(l1, l2, l3, minimizer = 'quadprog')
            gtv2 = GraphTotalVariance(l1, l2, l3, minimizer = 'lbfgs')
            t1 = time.time()
            gtv1.fit(X, y, cov)
            print(time.time() - t1)
            t2 = time.time()
            gtv2.fit(X, y, cov)
            print(time.time() - t2)
#            gtv1.coef_[gtv1.coef_ < 1e-6] = 0
#            gtv2.coef_[gtv2.coef_ < 1e-6] = 0
            pdb.set_trace()
            assert(np.allclose(gtv1.coef_, gtv2.coef_))

0.0447077751159668
0.016701698303222656
> <ipython-input-6-c03cd953e2ae>(24)<module>()
-> assert(np.allclose(gtv1.coef_, gtv2.coef_))
(Pdb) gtv1.coef_
array([ 1.7271015 , -0.0772191 ,  2.14700662,  0.14314027, -1.20474065,
        0.49656908, -0.11904619, -0.92043194,  1.61865897,  3.9916289 ,
        1.12575461,  6.34529609, -3.39891335,  2.52380212,  0.05810623,
        6.14461331,  4.94994924, -3.89084659, -0.76551383,  3.31288606,
        8.83176942,  0.79117917,  0.87422084, -0.93690427,  0.53848065,
       -0.83883048,  6.80555759,  3.67806159,  4.92131488, -2.29746997])
(Pdb) gtv2.coef_
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
